In [ ]:
%cd /content/drive/MyDrive/Agriculture App/agriculture-predictor-planner

/content/drive/MyDrive/Agriculture App/agriculture-predictor-planner


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
crop_data = pd.read_csv('data/merged/crop_soil_weather_merged.csv')
print(crop_data.head())


In [ ]:
pd.set_option('display.max_columns', None)
crop_data.info(verbose=True)

In [ ]:
#Converting data types so that district code, year, and state code are not treated as numeric during calculations
crop_data = crop_data.astype({'Dist Code':'category', 'Year':'category', 'State Code':'category', 'State Name':'category', 'Dist Name':'category'})

print(crop_data.info(verbose=True))

In [ ]:
crop_data.to_csv('crop_soil_weather_merged.csv', index=False)

In [ ]:
numeric_cols = crop_data.select_dtypes(include='number').columns
num_cols = len(numeric_cols)

# Set grid dimensions (e.g., 3 plots per row)
n_cols = 3
n_rows = (num_cols + n_cols - 1) // n_cols  # Round up to determine number of rows

# Create a large figure for all histograms
plt.figure(figsize=(n_cols * 5, n_rows * 4))

# Loop over numeric columns and create a histogram for each
for i, col in enumerate(numeric_cols, 1):
    plt.subplot(n_rows, n_cols, i)
    plt.hist(crop_data[col].dropna(), bins=30, edgecolor='black')  # Drop NA values for clean plotting
    plt.title(col)

plt.tight_layout()  # Adjust layout to prevent overlapping
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
crop_data[['RICE AREA (1000 ha)']].boxplot()
plt.show()

In [ ]:
sns.pairplot(crop_data)